In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_groq import ChatGroq
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts import MessagesPlaceholder, ChatPromptTemplate
from langchain.prompts import PromptTemplate
from langchain_core.chat_history import BaseChatMessageHistory
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import ConversationalRetrievalChain, LLMChain
from langchain.chains.history_aware_retriever import create_history_aware_retriever
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ConversationBufferMemory
from dotenv import load_dotenv
import os

In [175]:
load_dotenv()
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')
groq_api = os.getenv('GROQ_API_KEY')


In [ ]:
llm = ChatGroq(model = "llama-3.3-70b-versatile",
               groq_api_key=os.getenv('GROQ_API_KEY'))


In [177]:
## doc preprocessing

txt = TextLoader('FAQ.txt').load()
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(txt)
embedding= HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')
vector_store = Chroma.from_documents(documents=chunks, embedding=embedding)

In [178]:
q_prompt_template = ChatPromptTemplate.from_messages([
    ("system", 
     "Given the chat history and the user's latest question, "
     "rephrase the latest question as a standalone question that can be answered independently."),
    ("user", 
     "Chat history:\n{chat_history}\n\nUser question:\n{question}")
])

qa_prompt_template = ChatPromptTemplate.from_messages([
    ("system", 
     "You are an institute FAQ bot. Answer the question based on the context below. "
     "You can give the additional information if required but make sure it is related with the asked question. "
     "If answer is not present in context, say 'I don't know, would you like me to connect you to a human?'"),
    ("user", 
     "Context:\n{context}\n\nQuestion:\n{question}")
])

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key="answer")

# 2️⃣ Basic retriever
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

# 3️⃣ Conversational Retrieval Chain with query rewriting
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    return_source_documents=True,
    condense_question_prompt=q_prompt_template,  # <--- here you pass your query-rewriting prompt
    combine_docs_chain_kwargs={"prompt": qa_prompt_template}  # answer generation
)

In [179]:
result = qa_chain.invoke({"question": "What documents are required for admission and how do i get it?"})
print("Answer:", result["answer"])


Answer: The required documents for admission are scanned copies of your:

1. 10th Grade Marksheet and Certificate
2. 12th Grade Marksheet and Certificate

You can obtain these documents from your previous schools or educational institutions where you completed your 10th and 12th grades. Typically, you can get them by contacting the school administration or visiting the institution in person. They will provide you with the marksheet and certificate, which you can then scan and upload during the application process. 

To apply, you need to submit these documents online through the institute's official admissions portal. You will need to register, fill out the application form, upload the required documents, and pay the application fee.


In [180]:
result = qa_chain.invoke({'question': 'wont i need adharcard for it?'})

In [181]:
print(result['answer'])

You will need a Government-issued Photo ID, and an Aadhaar Card is an example of an acceptable document. You can also use a Passport as an alternative.


In [182]:
result = qa_chain.invoke({'question': 'why didnt you mention aadharcard and photos and all in the first question?'})
print(result['answer'])

I don't know, would you like me to connect you to a human?
